# Yammer Database Creation for AB Testing
Yammer has provided 4 different csv files that detail email logs sent to users ('YAMMER_EMAILS.csv'), user activity/event logs ('YAMMER_EVENTS.csv'), the status of membership of users involved in the AB test ('YAMMER_EXPERIMENTS.csv'), and the status of account activation of users ('YAMMER_USERS.csv'). Within this notebook, I will be creating a database from these csv files so I will be able to query data from it when performing analytics in the next notebook.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

pd.set_option('max_columns', 180)
pd.set_option('max_rows', 2000000)
pd.set_option('max_colwidth', 5000)

csvs = ['YAMMER_EMAILS.csv', 'YAMMER_EVENTS.csv', 'YAMMER_EXPERIMENTS.csv', 'YAMMER_USERS.csv']

df_list = []

for i in np.arange(0,len(csvs)):
    df_list.append(pd.read_csv("datasets/yammer/{}".format(csvs[i]),low_memory=False))
     
for d in np.arange(0,len(df_list)):
    print('#########################{}#################################'.format(csvs[d]))
    print(df_list[d].shape)
    print(df_list[d].head(2))
    print('############################################################################')

emails = df_list[0]
events = df_list[1]
experiments = df_list[2]
users = df_list[3]


#########################YAMMER_EMAILS.csv#################################
(90389, 4)
   user_id          occurred_at              action  user_type
0      0.0  2014-05-06 09:30:00  sent_weekly_digest        1.0
1      0.0  2014-05-13 09:30:00  sent_weekly_digest        1.0
############################################################################
#########################YAMMER_EVENTS.csv#################################
(340832, 7)
   user_id          occurred_at  event_type event_name location  \
0  10522.0  2014-05-02 11:02:39  engagement      login    Japan   
1  10522.0  2014-05-02 11:02:53  engagement  home_page    Japan   

                   device  user_type  
0  dell inspiron notebook        3.0  
1  dell inspiron notebook        3.0  
############################################################################
#########################YAMMER_EXPERIMENTS.csv#################################
(2595, 7)
   user_id          occurred_at        experiment experiment_group locat

In [3]:
DB = 'yammer.db'

def run_query(q):
    with sqlite3.connect(DB) as conn:
        return pd.read_sql(q,conn)

def run_command(c):
    with sqlite3.connect(DB) as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)

def show_tables():
    q = '''
    SELECT
        name,
        type
    FROM sqlite_master
    WHERE type IN ("table","view");
    '''
    return run_query(q)

show_tables()

,name,type
0,y_users,table
1,y_experiments,table
2,y_events,table
3,y_emails,table


In [206]:
tables = {
    "emails": emails,
    "events": events,
    "experiments": experiments,
    "users": users
}

with sqlite3.connect(DB) as conn:    
    for name, data in tables.items():
        conn.execute("DROP TABLE IF EXISTS {};".format(name))
        data.to_sql(name,conn,index=False)
show_tables()

,name,type
0,emails,table
1,events,table
2,experiments,table
3,users,table


All tables have user_id as primary key. Will create schema and link emails, events, experiments to users table.

In [207]:
## create user table
c1 = """
CREATE TABLE IF NOT EXISTS y_users (
    user_id INTEGER PRIMARY KEY,
    created_at DATETIME,
    company_id INTEGER,
    language TEXT,
    activated_at DATETIME,
    state TEXT
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO y_users
SELECT
    user_id,
    created_at,
    company_id,
    language,
    activated_at,
    state
FROM users;
"""

q = """
SELECT * FROM users
ORDER BY 1
LIMIT 5
;
"""

run_command(c1)
run_command(c2)
run_query(q)

,user_id,created_at,company_id,language,activated_at,state
0,0.0,2013-01-01 20:59:39,5737.0,english,2013-01-01 21:01:07,active
1,1.0,2013-01-01 13:07:46,28.0,english,None,pending
2,2.0,2013-01-01 10:59:05,51.0,english,None,pending
3,3.0,2013-01-01 18:40:36,2800.0,german,2013-01-01 18:42:02,active
4,4.0,2013-01-01 14:37:51,5110.0,indian,2013-01-01 14:39:05,active


In [208]:
## verify that schema of old and new user tables are different
q =  """PRAGMA table_info(y_users);"""
run_query(q)

,cid,name,type,notnull,dflt_value,pk
0,0,user_id,INTEGER,0,None,1
1,1,created_at,DATETIME,0,None,0
2,2,company_id,INTEGER,0,None,0
3,3,language,TEXT,0,None,0
4,4,activated_at,DATETIME,0,None,0
5,5,state,TEXT,0,None,0


In [209]:
## create experiment table
c1 = """
CREATE TABLE IF NOT EXISTS y_experiments (
    user_id INTEGER PRIMARY KEY,
    occurred_at DATETIME,
    experiment TEXT,
    experiment_group TEXT,
    device TEXT,
    user_type INTEGER,
    FOREIGN KEY (user_id) REFERENCES y_users(user_id)
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO y_experiments
SELECT
    user_id,
    occurred_at,
    experiment,
    experiment_group,
    device,
    user_type
FROM experiments;
"""

q = """
SELECT * FROM y_experiments
ORDER BY 1
LIMIT 5
;
"""

run_command(c1)
run_command(c2)
run_query(q)

,user_id,occurred_at,experiment,experiment_group,device,user_type
0,4,2014-06-05 15:20:16,publisher_update,control_group,lenovo thinkpad,3
1,11,2014-06-17 09:31:22,publisher_update,control_group,iphone 4s,1
2,19,2014-06-04 09:31:33,publisher_update,test_group,iphone 5,1
3,20,2014-06-18 09:31:29,publisher_update,control_group,samsung galaxy s4,2
4,22,2014-06-02 10:57:04,publisher_update,test_group,lenovo thinkpad,1


In [210]:
## create events table
## removed primary key code because w/ pk, lost 9 values in table:     
# PRIMARY KEY (user_id, occurred_at, event_name),
# FOREIGN KEY (user_id) REFERENCES y_users(user_id)

c1 = """
CREATE TABLE IF NOT EXISTS y_events (
    user_id INTEGER,
    occurred_at DATETIME,
    event_type TEXT,
    event_name TEXT,
    location TEXT,
    device TEXT,
    user_type INTEGER

);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO y_events
SELECT
    user_id,
    occurred_at,
    event_type,
    event_name,
    location,
    device,
    user_type
FROM events;
"""

q = """
SELECT * FROM y_events
ORDER BY 1
LIMIT 5
;
"""

run_command(c1)
run_command(c2)
run_query(q)

,user_id,occurred_at,event_type,event_name,location,device,user_type
0,4,2014-05-13 09:31:47,engagement,login,India,lenovo thinkpad,3
1,4,2014-05-13 09:32:10,engagement,home_page,India,lenovo thinkpad,3
2,4,2014-05-13 09:32:26,engagement,search_autocomplete,India,lenovo thinkpad,3
3,4,2014-05-13 09:32:58,engagement,search_autocomplete,India,lenovo thinkpad,3
4,4,2014-05-24 11:39:53,engagement,login,India,samsung galaxy s4,3


In [211]:
## verify that schema of old and new events tables are different
q =  """PRAGMA table_info(y_events);"""
run_query(q)

,cid,name,type,notnull,dflt_value,pk
0,0,user_id,INTEGER,0,None,0
1,1,occurred_at,DATETIME,0,None,0
2,2,event_type,TEXT,0,None,0
3,3,event_name,TEXT,0,None,0
4,4,location,TEXT,0,None,0
5,5,device,TEXT,0,None,0
6,6,user_type,INTEGER,0,None,0


In [212]:
## create email table
c1 = """
CREATE TABLE IF NOT EXISTS y_emails (
    user_id INTEGER,
    occurred_at DATETIME,
    action TEXT,
    user_type INTEGER,
    PRIMARY KEY (user_id, occurred_at),
    FOREIGN KEY (user_id) REFERENCES y_users(user_id)
);
"""
## add data from csv
c2 = """
INSERT OR IGNORE INTO y_emails
SELECT
    user_id,
    occurred_at,
    action,
    user_type
FROM emails;
"""

q = """
SELECT * FROM y_emails
ORDER BY 1
LIMIT 5
;
"""

run_command(c1)
run_command(c2)
run_query(q)

,user_id,occurred_at,action,user_type
0,0,2014-05-06 09:30:00,sent_weekly_digest,1
1,0,2014-05-13 09:30:00,sent_weekly_digest,1
2,0,2014-05-20 09:30:00,sent_weekly_digest,1
3,0,2014-05-27 09:30:00,sent_weekly_digest,1
4,0,2014-06-03 09:30:00,sent_weekly_digest,1


In [213]:
q =  """PRAGMA table_info(y_emails);"""
run_query(q)

,cid,name,type,notnull,dflt_value,pk
0,0,user_id,INTEGER,0,None,1
1,1,occurred_at,DATETIME,0,None,2
2,2,action,TEXT,0,None,0
3,3,user_type,INTEGER,0,None,0


In [217]:
#remove original tables
tables = [
    "emails",
    "events",
    "users",
    "experiments"
]

for t in tables:
    c = '''
    DROP TABLE {}
    '''.format(t)
    
    run_command(c)

show_tables()

,name,type
0,y_users,table
1,y_experiments,table
2,y_events,table
3,y_emails,table


In [215]:
# sample queries
q = """
    SELECT eve.*
    FROM y_experiments exp
    INNER JOIN y_events eve
    ON exp.user_id = eve.user_id
    WHERE eve.occurred_at <= '2014-07-01' AND eve.occurred_at >= '2014-06-01'
    ORDER BY eve.occurred_at DESC

"""
run_query(q).shape

(81200, 7)

In [219]:
q = """
    SELECT *
    FROM y_users
    ;

"""
run_query(q).head()

,user_id,created_at,company_id,language,activated_at,state
0,0,2013-01-01 20:59:39,5737,english,2013-01-01 21:01:07,active
1,1,2013-01-01 13:07:46,28,english,None,pending
2,2,2013-01-01 10:59:05,51,english,None,pending
3,3,2013-01-01 18:40:36,2800,german,2013-01-01 18:42:02,active
4,4,2013-01-01 14:37:51,5110,indian,2013-01-01 14:39:05,active
